In [1]:
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML,display,Image
display(HTML('<h3>Which countries/states are improving and which are getting worse?</h3><h4>The trend is determined by the last 14 days new cases trend.</h4><h3></h3>'))
display(HTML('<a href="http://www.cov2response.com"> Back to cov2response.com </a>'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
g=!ls ../csse_covid_19_daily_reports/*.csv
al=[]
for f in g:
    if pd.to_datetime(f.split('/')[-1][:10]) > pd.to_datetime('2020-03-25'):      
        d=pd.read_csv(f)
        d['dt']=pd.to_datetime(f.split('/')[-1][:10])
        al.append(d)
adt=pd.concat(al)
adt.fillna('',inplace=True)
adt['State_Country']=adt['Country_Region']+ ' '+ adt['Province_State']
f=adt.groupby(by=['State_Country','dt']).sum().reset_index()
us_only=adt[adt.Country_Region=='US'].groupby(by=['Country_Region','dt']).sum().reset_index().copy()
us_only.columns= ['State_Country',         'dt',  'Confirmed',  'Deaths',  'Recovered',  'Active']
f=pd.concat([f,us_only],axis=0).reset_index()
del(f['index'])
mydict={}
for i in f.State_Country.unique():
    if 'Spain' in i:
        continue
        
    mydf=f[f.State_Country == i].copy()
    if len(mydf)<14:
        continue
    mydf['NewCases']=mydf.Confirmed.diff()
    if mydf['NewCases'].mean() <100:
        continue
    mydf=mydf.fillna(mydf.NewCases.min())
    
    mydf['NewCases2']=mydf.NewCases.rolling(7).mean()
    
    #plt.plot(mydf['NewCases'])
    #plt.figure(figsize=(10,5))
    #plt.plot(mydf['NewCases2'])
    last_points=mydf['NewCases2'][-14:]
    slope = np.polyfit(range(14),last_points,1)[0]
    if slope > 0:
        state='Getting Worse'
    else:
        state='Improving'
    mydict[i]={'slope':slope,'max':mydf['NewCases2'].max(),#'last_points':last_points,\
               'all_points':mydf['NewCases2'].dropna().values}

ft=pd.DataFrame(mydict).T
ft['Type']=ft.slope.map( lambda x: 'Improving' if x < -1 else 'Getting Worse' if x > 1 else 'About Same')


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import matplotlib as mpl
import matplotlib.cm as cm
norm = mpl.colors.Normalize(vmin=0, vmax=2000)
cmap = plt.cm.Blues
m = cm.ScalarMappable(norm=norm, cmap=cmap)

def sparkline(data, figsize=(5, 3), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    
    data = list(data)
    
 
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.set_ylim(0,max(data)*1.2)
    ax.plot(data)
    
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylim(0,max(data)*1.2)
    
 
    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')
    plt.text(len(data) - 1, data[len(data) - 1], np.round(data[len(data) - 1],-1).astype(int))
 
    #ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.8, color=m.to_rgba(np.mean(data)) )
    ax.fill_between(range(len(data)), data, len(data)*[0], alpha=0.8, color=m.to_rgba(np.mean(data)) )
 
    img = BytesIO()
    plt.savefig(img, transparent=True, bbox_inches='tight')
    img.seek(0)
    plt.close()
 
    return base64.b64encode(img.read()).decode("UTF-8")

pd.set_option('display.max_colwidth', -1)
#p.loc[:,['fig']].style
#p.loc[:,['fig']].style

In [ ]:
from IPython.core.display import HTML,display,Image
display(HTML('<h4>US States </h4>'))
t='Getting Worse'
p=ft[ ((ft.Type==t) & ( ft.index.str.contains('US '))) ].sort_values(by='slope',ascending=False ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
a=p.loc[:,[t]].reset_index().copy()
a.columns=[f'{t} Country/State',f'New Cases/day']
#display(HTML(a.to_html(escape=False)))
#display(HTML(p.loc[:,[t]].to_html(escape=False)))
#print(p.slope)

t='Improving'
p=ft[((ft.Type==t) & ( ft.index.str.contains('US ')))].sort_values(by='slope',ascending=True ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
b=p.loc[:,[t]].reset_index().copy()
b.columns=[f'{t} Country/State',f'NewCases/day']

#display(HTML(p.loc[:,[t]].to_html(escape=False)))
t='About Same'
p=ft[((ft.Type==t) & ( ft.index.str.contains('US ')))].sort_values(by='slope',ascending=True ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
c=p.loc[:,[t]].reset_index().copy()
c.columns=[f'{t} Country/State',f'newcases per day']
#display(HTML(p.loc[:,[t]].to_html(escape=False)))

pd.concat([a,b,c],axis=1).fillna('').style.hide_index().set_properties(**{
    'font-size': '15pt',
})

In [ ]:
from IPython.core.display import HTML,display,Image
display(HTML('<h4>Global </h4>'))
t='Getting Worse'
p=ft[ ((ft.Type==t) & (~ ft.index.str.contains('US '))) ].sort_values(by='slope',ascending=False ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
a=p.loc[:,[t]].reset_index().copy()
a.columns=[f'{t} Country/State',f'New Cases/day']
#display(HTML(a.to_html(escape=False)))
#display(HTML(p.loc[:,[t]].to_html(escape=False)))
#print(p.slope)

t='Improving'
p=ft[((ft.Type==t) & ( ~ft.index.str.contains('US ')))].sort_values(by='slope',ascending=True ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
b=p.loc[:,[t]].reset_index().copy()
b.columns=[f'{t} Country/State',f'NewCases/day']

#display(HTML(p.loc[:,[t]].to_html(escape=False)))
t='About Same'
p=ft[((ft.Type==t) & ( ~ft.index.str.contains('US ')))].sort_values(by='slope',ascending=True ).copy()
p[t]=p.all_points.map( lambda x: '<img src="data:image/png;base64,{}"/>'.format(sparkline(x)))
c=p.loc[:,[t]].reset_index().copy()
c.columns=[f'{t} Country/State',f'newcases per day']
#display(HTML(p.loc[:,[t]].to_html(escape=False)))
pd.concat([a,b,c],axis=1).fillna('').style.hide_index().set_properties(**{
    'font-size': '15pt',
})

In [ ]:
from IPython.core.display import HTML,display,Image
display(HTML('<a href="http://www.cov2response.com"> Back to cov2response.com </a>'))
print(f'Generated on {datetime.datetime.now()}')